In [1]:
# gpu number setting
import os
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ["CUDA_VISIBLE_DEVICES"] = '1' ## gpu 번호 셋팅 윤건 :0, 기준 : 1, 준형 :2,

# Gpu growth setting
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)


# tensorflow & keras version check
print('tensorflow version : ' , tf.__version__)
print('keras version : ' , tf.keras.__version__)

# tensorflow gpu available check 
print('GPU available ? : ', tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None))

tensorflow version :  2.3.1
keras version :  2.4.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU available ? :  True


In [17]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

from deepctr.models import *
from deepctr.feature_column import SparseFeat, DenseFeat, get_feature_names

In [3]:
pd.set_option('display.max_columns', 100)

In [4]:
pd.set_option('display.max_rows', 100)

In [5]:
raw = pd.read_csv('../Data/tb_ipr_m_seamless_2nd_movie.txt', sep='', header=None)

/home/ubuntu/anaconda3/envs/tensorflow2_latest_p37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (0,25,26,27,28) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
raw.columns = ['tb_ipr_m_seamless_2nd_movie.sa_id',
       'tb_ipr_m_seamless_2nd_movie.s_time',
       'tb_ipr_m_seamless_2nd_movie.e_time',
       'tb_ipr_m_seamless_2nd_movie.watch_duration',
       'tb_ipr_m_seamless_2nd_movie.album_id',
       'tb_ipr_m_seamless_2nd_movie.album_name',
       'tb_ipr_m_seamless_2nd_movie.vod_s_point',
       'tb_ipr_m_seamless_2nd_movie.vod_e_point',
       'tb_ipr_m_seamless_2nd_movie.agree_yn',
       'tb_ipr_m_seamless_2nd_movie.category_id',
       'tb_ipr_m_seamless_2nd_movie.release_date',
       'tb_ipr_m_seamless_2nd_movie.run_time',
       'tb_ipr_m_seamless_2nd_movie.meta_genre',
       'tb_ipr_m_seamless_2nd_movie.genre_large',
       'tb_ipr_m_seamless_2nd_movie.genre_mid',
       'tb_ipr_m_seamless_2nd_movie.genre_small',
       'tb_ipr_m_seamless_2nd_movie.movie_meta_price',
       'tb_ipr_m_seamless_2nd_movie.buy_history_price',
       'tb_ipr_m_seamless_2nd_movie.i30_meta_price',
       'tb_ipr_m_seamless_2nd_movie.buy_tot',
       'tb_ipr_m_seamless_2nd_movie.view_no',
       'tb_ipr_m_seamless_2nd_movie.fod',
       'tb_ipr_m_seamless_2nd_movie.buy_1_2',
       'tb_ipr_m_seamless_2nd_movie.buy_3',
       'tb_ipr_m_seamless_2nd_movie.buy_seg',
       'tb_ipr_m_seamless_2nd_movie.amt_1_4',
       'tb_ipr_m_seamless_2nd_movie.amt_2_4',
       'tb_ipr_m_seamless_2nd_movie.amt_3_4',
       'tb_ipr_m_seamless_2nd_movie.amt_4_4',
       'tb_ipr_m_seamless_2nd_movie.amt_r_gabun',
       'tb_ipr_m_seamless_2nd_movie.weekdays',
       'tb_ipr_m_seamless_2nd_movie.weekends',
       'tb_ipr_m_seamless_2nd_movie.dawn',
       'tb_ipr_m_seamless_2nd_movie.morning',
       'tb_ipr_m_seamless_2nd_movie.afternoon',
       'tb_ipr_m_seamless_2nd_movie.evening',
       'tb_ipr_m_seamless_2nd_movie.watch_ratio',
       'tb_ipr_m_seamless_2nd_movie.current_rate',
       'tb_ipr_m_seamless_2nd_movie.ncf_rating']

In [7]:
raw.columns = list(map(lambda x: x.split('.')[-1], raw.columns))

In [ ]:
set(test['buy_history_price'])

In [42]:
test[test['amt_2_4'].apply(lambda x: x not in ['0.0', '1.0', 0, 1, '\\N'])]

,sa_id,s_time,e_time,watch_duration,album_id,album_name,vod_s_point,vod_e_point,agree_yn,category_id,release_date,run_time,meta_genre,genre_large,genre_mid,genre_small,movie_meta_price,buy_history_price,i30_meta_price,buy_tot,view_no,fod,buy_1_2,buy_3,buy_seg,amt_1_4,amt_2_4,amt_3_4,amt_4_4,amt_r_gabun,weekdays,weekends,dawn,morning,afternoon,evening,watch_ratio,current_rate,ncf_rating
228,206073321523,20200607210646,20200607210827,100.0,M0119AM413PPV00,퍼펙트맨,384,484,Y,NaN,20191002.0,116,코미디,영화,코미디,NaN,2500.0,\N,2500,3,0,0,0,1,BUY_3,0,0.666667,0.333333,0,AMT_0_2500_R,0,1,0,0,0,1,0.014368,0.833333,0.090517
229,206073321523,20200603204442,20200603204931,286.0,M0119AM413PPV00,퍼펙트맨,84,370,Y,NaN,20191002.0,116,코미디,영화,코미디,NaN,2500.0,\N,2500,3,0,0,0,1,BUY_3,0,0.666667,0.333333,0,AMT_0_2500_R,1,0,0,0,0,1,0.041092,0.855556,0.092768
230,206073321523,20200515211145,20200515214650,2094.0,M01131EA28PPV00,도마뱀,38,2132,Y,NaN,20060427.0,117,로맨스,영화,로맨스,NaN,2500.0,\N,2500,3,0,0,0,1,BUY_3,0,0.666667,0.333333,0,AMT_0_2500_R,1,0,0,0,0,1,0.298291,0.961111,0.168590
231,206073321523,20200516152325,20200516163334,4181.0,M01131EA28PPV00,도마뱀,2748,6929,Y,NaN,20060427.0,117,로맨스,영화,로맨스,NaN,2500.0,\N,2500,3,0,0,0,1,BUY_3,0,0.666667,0.333333,0,AMT_0_2500_R,0,1,0,0,1,0,0.595584,0.955556,0.320014
232,206073321523,20200626105504,20200626110010,300.0,M01131MA88PPV00,더 클럽,2327,2627,Y,NaN,20151015.0,107,로맨스,영화,로맨스,NaN,0.0,\N,0,3,0,0,0,1,BUY_3,0,0.666667,0.333333,0,AMT_0_2500_R,1,0,0,1,0,0,0.046729,0.727778,0.159476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6759108,M17022300005,20200720112220,20200720112327,67.0,M012077425PPV00,결백,0,67,Y,NaN,20200610.0,110,드라마,영화,드라마,NaN,7500.0,15000,7500,6,0,0,0,1,BUY_3,0,0.333333,0,0.666667,AMT_5000_OVER_R,1,0,0,1,0,0,0.010152,0.594444,0.207854
6759109,M17022300005,20200527154542,20200527155041,293.0,M011861007PPV00,"안녕, 나의 소녀",1,294,Y,NaN,20180516.0,104,로맨스,영화,로맨스,NaN,1400.0,1400,1400,6,0,0,0,1,BUY_3,0,0.333333,0,0.666667,AMT_5000_OVER_R,1,0,0,0,1,0,0.046955,0.894444,0.076255
6759110,M17022300005,20200605110204,20200605110217,12.0,M012057125PPV00,어쩌다 로맨스,1000,1012,Y,NaN,20200101.0,88,로맨스,영화,로맨스,NaN,2500.0,5000,2500,6,0,0,0,1,BUY_3,0,0.333333,0,0.666667,AMT_5000_OVER_R,1,0,0,1,0,0,0.002273,0.844444,0.078914
6759111,M17022300005,20200605110247,20200605110314,26.0,M012057125PPV00,어쩌다 로맨스,1010,1036,Y,NaN,20200101.0,88,로맨스,영화,로맨스,NaN,2500.0,5000,2500,6,0,0,0,1,BUY_3,0,0.333333,0,0.666667,AMT_5000_OVER_R,1,0,0,1,0,0,0.004924,0.844444,0.080240


In [22]:
sum(raw['agree_yn'] == 'Y')

5375039

In [11]:
raw['category_id'].tail(100)

6759093      NaN
6759094      NaN
6759095      NaN
6759096      NaN
6759097      NaN
6759098      NaN
6759099      NaN
6759100      NaN
6759101      NaN
6759102      NaN
6759103      NaN
6759104      NaN
6759105      NaN
6759106      NaN
6759107      NaN
6759108      NaN
6759109      NaN
6759110      NaN
6759111      NaN
6759112      NaN
6759113      NaN
6759114      NaN
6759115      NaN
6759116      NaN
6759117      NaN
6759118      NaN
6759119      NaN
6759120      NaN
6759121      NaN
6759122      NaN
6759123      NaN
6759124      NaN
6759125      NaN
6759126      NaN
6759127      NaN
6759128      NaN
6759129      NaN
6759130      NaN
6759131      NaN
6759132      NaN
6759133      NaN
6759134      NaN
6759135      NaN
6759136      NaN
6759137      NaN
6759138      NaN
6759139      NaN
6759140      NaN
6759141      NaN
6759142      NaN
6759143      NaN
6759144      NaN
6759145      NaN
6759146      NaN
6759147      NaN
6759148      NaN
6759149      NaN
6759150      NaN
6759151      N

In [12]:
raw.columns

Index(['sa_id', 's_time', 'e_time', 'watch_duration', 'album_id', 'album_name',
       'vod_s_point', 'vod_e_point', 'agree_yn', 'category_id', 'release_date',
       'run_time', 'meta_genre', 'genre_large', 'genre_mid', 'genre_small',
       'movie_meta_price', 'buy_history_price', 'i30_meta_price', 'buy_tot',
       'view_no', 'fod', 'buy_1_2', 'buy_3', 'buy_seg', 'amt_1_4', 'amt_2_4',
       'amt_3_4', 'amt_4_4', 'amt_r_gabun', 'weekdays', 'weekends', 'dawn',
       'morning', 'afternoon', 'evening', 'watch_ratio', 'current_rate',
       'ncf_rating'],
      dtype='object')

* distinct user 968,091
* distinct album 10,570

* featrue 분리, sparse, dense, ambiguous, unnecessary

* meta_genre, 대,중,소장르의 관계
* target column을 뭘로할지
* how to sample negatively?

In [13]:
sparse_features = ['sa_id', 'album_id', 'category_id', 'buy_seg']
dense_features = ['release_date', 'run_time', 'movie_meta_price', 'buy_history_price', 'i30_meta_price', 'buy_tot',
                  'amt_1_4', 'amt_2_4', 'amt_3_4', 'amt_4_4']
ambiguous_features = [ 'agree_yn', 'meta_genre', 'genre_large', 'genre_mid', 'genre_small']
unnecessary_features = ['s_time', 'e_time', 'watch_duration', 'vod_s_point', 'vod_e_point', 'album_name',
                        'view_no', 'fod', 'buy_1_2', 'buy_3', 'amt_r_gabun', 'watch_ratio',
                        'weekdays', 'weekends', 'dawn', 'morning', 'afternoon', 'evening', 'current_rate', 'ncf_rating']
target = []

In [43]:
user_col = ['sa_id', 'buy_seg', 'buy_tot', 'amt_1_4', 'amt_2_4', 'amt_3_4', 'amt_4_4']
cont_col = ['album_id', 'category_id', 'release_date', 'run_time', 'movie_meta_price', 'buy_history_price', 'i30_meta_price', 'meta_genre', 'genre_large', 'genre_mid', 'genre_small']

In [14]:
print(len(raw.columns))
print(len(sparse_features) +len(dense_features)+ len(ambiguous_features) + len(unnecessary_features) + len(target))

39
39


In [8]:
df_1 = raw[['sa_id', 'album_id', 'ncf_rating']]

In [9]:
df_1

,sa_id,album_id,ncf_rating
0,205101491713,M01198J812PPV00,0.493201
1,205112221633,M011449172PPV00,0.062150
2,205112221633,M0114CB383PPV00,0.048566
3,205112221633,M0111CMB08PPV00,0.054389
4,205112403631,M0119A8236PPV00,0.657391
...,...,...,...
6759188,M18061900001,M011849006PPV00,0.560919
6759189,M18061900001,M01203Q384PPV00,0.220670
6759190,M18061900001,M01206H267PPV00,0.606863
6759191,M18061900001,M01206U088PPV00,0.265359


In [57]:
df_1[df_1['ncf_rating'].isnull()]

,sa_id,album_id,ncf_rating


In [10]:
data = df_1

In [11]:
sparse_features = ['sa_id', 'album_id']
target = ['ncf_rating']

In [12]:
data[['sa_id', 'album_id']] = data[['sa_id', 'album_id']].astype(str)

/home/ubuntu/anaconda3/envs/tensorflow2_latest_p37/lib/python3.7/site-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [13]:
for feat in sparse_features:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])

/home/ubuntu/anaconda3/envs/tensorflow2_latest_p37/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [69]:
data

,sa_id,album_id,ncf_rating
0,68,8282,0.493201
1,103,1724,0.062150
2,103,2542,0.048566
3,103,257,0.054389
4,114,8512,0.657391
...,...,...,...
6759188,968066,6451,0.560919
6759189,968066,9625,0.220670
6759190,968066,10308,0.606863
6759191,968066,10376,0.265359


In [14]:
fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=data[feat].nunique(),embedding_dim=4)
                           for i,feat in enumerate(sparse_features)]

dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [15]:
train, test = train_test_split(data, test_size=0.2, random_state=2020)
train_model_input = {name:train[name] for name in feature_names}
test_model_input = {name:test[name] for name in feature_names}

In [72]:
train

,sa_id,album_id,ncf_rating
1383350,207451,4722,0.495666
4229202,80729,9697,0.182301
6105258,135178,5852,0.669257
2668662,842190,8183,0.145488
3996905,673129,1004,0.057859
...,...,...,...
1508989,948238,1108,0.608333
4737115,97494,9498,0.390180
1948278,574915,10269,0.204455
1482051,770578,2053,0.567666


In [73]:
test

,sa_id,album_id,ncf_rating
2277460,546996,10466,0.688175
2137389,695987,6236,0.473968
317118,839202,7654,0.400527
3066251,189539,3226,0.097643
6018858,602581,8550,0.422583
...,...,...,...
3257262,306624,4241,0.017178
2496061,837410,29,0.121685
2889854,148488,3346,0.173759
2666161,829010,8188,0.600000


In [16]:
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='regression')
model.compile("adam", "mse", metrics=['mse'], )

history = model.fit(train_model_input, train[target].values,
                    batch_size=1024, epochs=10, verbose=2, validation_split=0.2, )
pred_ans = model.predict(test_model_input, batch_size=32)
print("test MSE", round(mean_squared_error(
    test[target].values, pred_ans), 4))

Epoch 1/10


/home/ubuntu/anaconda3/envs/tensorflow2_latest_p37/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


4225/4225 - 55s - loss: 0.0396 - mse: 0.0353 - val_loss: 0.0395 - val_mse: 0.0341
Epoch 2/10
4225/4225 - 54s - loss: 0.0417 - mse: 0.0334 - val_loss: 0.0429 - val_mse: 0.0339
Epoch 3/10
4225/4225 - 54s - loss: 0.0417 - mse: 0.0319 - val_loss: 0.0446 - val_mse: 0.0343
Epoch 4/10
4225/4225 - 54s - loss: 0.0430 - mse: 0.0322 - val_loss: 0.0456 - val_mse: 0.0342
Epoch 5/10
4225/4225 - 54s - loss: 0.0435 - mse: 0.0321 - val_loss: 0.0457 - val_mse: 0.0343
Epoch 6/10
4225/4225 - 54s - loss: 0.0427 - mse: 0.0318 - val_loss: 0.0450 - val_mse: 0.0341
Epoch 7/10
4225/4225 - 54s - loss: 0.0429 - mse: 0.0318 - val_loss: 0.0458 - val_mse: 0.0343
Epoch 8/10
4225/4225 - 54s - loss: 0.0435 - mse: 0.0320 - val_loss: 0.0459 - val_mse: 0.0341
Epoch 9/10
4225/4225 - 54s - loss: 0.0432 - mse: 0.0317 - val_loss: 0.0454 - val_mse: 0.0341
Epoch 10/10
4225/4225 - 54s - loss: 0.0427 - mse: 0.0316 - val_loss: 0.0458 - val_mse: 0.0345


NameError: name 'mean_squared_error' is not defined

In [18]:
print("test MSE", round(mean_squared_error(
    test[target].values, pred_ans), 4))

test MSE 0.0344
